In [ ]:
!pip install polars
#uncompress the kg
!tar -xvf ../output/monarch-kg.tar.gz -C ../output/    

In [11]:
%%time

import polars as pl
# read node and edge tsv files

nodes = pl.read_csv("../output/monarch-kg_nodes.tsv", separator="\t")
edges = pl.read_csv("../output/monarch-kg_edges.tsv", separator="\t")

closure = (pl.read_csv("../data/monarch/phenio-relation-filtered.tsv", 
                      separator="\t", 
                      has_header=False, 
                      new_columns=["entity","predicate","entity_closure"])
           .drop("predicate"))

closure_id = closure.group_by("entity").agg(pl.col("entity_closure"))
closure_label = closure.join(nodes.select(["id","name"]).rename({"name":"entity_closure_label"}), 
                                left_on="entity_closure", right_on="id", how="left").group_by("entity").agg(pl.col("entity_closure_label"))
# add namespace by splitting id column on : and taking the first element
nodes = nodes.with_columns([pl.col("id").str.split(":").map_elements(lambda s: s[0]).alias("namespace")])



CPU times: user 11.8 s, sys: 8.6 s, total: 20.4 s
Wall time: 10.7 s


In [ ]:
%%time

def add_node_attribute(field, attribute, edges, nodes):
    return edges.join(nodes.select(["id",attribute]).rename({attribute:f"{field}_{attribute}"}), 
               left_on=field, right_on="id", how="left")

def add_closure(field, edges, closure):
    return edges.join(closure.rename({"entity_closure":f"{field}_closure"}), 
               left_on=field, right_on="entity", how="left")

def add_closure_label(field, edges, closure_label):
    return edges.join(closure_label.rename({"entity_closure_label":f"{field}_closure_label"}), 
               left_on=field, right_on="entity", how="left")

def closurize(field, edges, nodes, closure):
    edges = add_closure(field, edges, closure)
    edges = add_closure_label(field, edges, closure_label)
    for attribute in ["namespace", "category"]:
        edges = add_node_attribute(field, attribute, edges, nodes)

    return edges
        
for field in ["subject", "object"]:
    edges = closurize(field, edges, nodes, closure)    
    
edges

In [ ]:
# add subject_category by joining subject to node id and getting the category column
edges.join(nodes.select(["id","category"]).rename({"category":"subject_category"}), left_on="subject", right_on="id", how="left")


In [ ]:
closure_id
